In [1]:
#installing the necessary libraries!
#!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

In [ ]:
import os
from langchain.document_loaders import UnstructuredFileLoader #to load documents
from langchain_text_splitters import CharacterTextSplitter # to chunk doc
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq # to load the llm
from langchain.chains import RetrievalQA

In [ ]:
os.environ['GROQ_API_KEY'] = "<your groq api key>"

In [ ]:
#loading the document
loader = UnstructuredFileLoader('/content/How to Win Friends & Influence People ( PDFDrive ).pdf')
documents = loader.load()

In [ ]:
#splitting into text chunks-
text_splitter = CharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 400
)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

306

In [ ]:
#texts[:4]

In [ ]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-8-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
persistant_directory = "doc_db" #directory for chromadb

In [ ]:

#store the vector embeddings!
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persistant_directory
)

In [ ]:
#creating a retreiver
retriever = vectordb.as_retriever()

In [ ]:
#load the llm from groq!
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.7
)

In [ ]:
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type ="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
#invoke the qa chain and get a response for user query
query ="When is it appropriate to admit your mistakes, and how can doing so strengthen relationships?"
response = qa_chain.invoke({"query": query})

In [ ]:
print(response['result'])

According to the text, it is appropriate to admit your mistakes quickly and emphatically, especially when you are wrong. This can strengthen relationships by:

1. Showing humility and a willingness to learn from errors, which can make others more receptive to constructive feedback.
2. Taking the fight out of the other person, making them less defensive and more open to finding a solution.
3. Creating a positive atmosphere, where both parties can work together to address the issue.
4. Showing respect for the other person's perspective and feelings, which can lead to increased trust and understanding.

Admitting mistakes can also have a positive impact on relationships by:

1. Clearing the air of guilt and defensiveness, making it easier to move forward.
2. Setting a good example, encouraging others to be honest and take responsibility for their own mistakes.
3. Showing that you value the relationship and are willing to take the high road, even when it's difficult.

Overall, admitting mi